In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from tabulate import tabulate

df_author = pd.read_json('lighter_authors.json',lines= True)

In [2]:
df_books = pd.read_json(r'lighter_books.json', lines=True, nrows=150000)

columns_to_drop = ['isbn', 'isbn13', 'asin', 'edition_information', 'publisher', 'series_id', 'series_name', 'series_position', 'description', 'shelves']

df_books.drop(columns=columns_to_drop, inplace=True)
df_books = df_books[df_books['title'].notna()]
df_books = df_books[df_books['ratings_count'] >= 0]
df_books = df_books[df_books['text_reviews_count'] >= 0]

# eleminate num_pages
df_books['num_pages'] = df_books['num_pages'].replace('', np.nan).astype(float)

# eleminate rows with num_pages less than 0
df_books = df_books[df_books['num_pages'] >= 0]

In [ ]:
# In order to avoid problems in terms of performance and visualization, we put range parameter in the codes. By removing this parameter, the same process can be applied for the whole dataset. 
# There is an extra work we did without range restriction below.

In [3]:
#RQ3 Human Code

# convert publication_date column to datetime format with an explicit format
df_books['publication_date'] = pd.to_datetime(df_books['publication_date'], format='%Y-%m-%d', errors='coerce')

def year_summary(year):
    
    # books published in the specified year
    year_books = df_books[df_books['publication_date'].dt.year == year]

    # the number of books published that year
    num_books_published = len(year_books)

    # number of pages written that year
    total_pages_written = year_books['num_pages'].sum()

    # group books by month and count the number of books published in each month
    month_counts = year_books['publication_date'].dt.month.value_counts()
    most_prolific_month = month_counts.idxmax()

    # longest book written that year
    longest_book = year_books.loc[year_books['num_pages'].idxmax()]

    return {
        'Year': year,
        'Number of Books Published': num_books_published,
        'Total Pages Written': total_pages_written,
        'Most Prolific Month': most_prolific_month,
        'Longest Book Title': longest_book['title'],
        'Longest Book Author': longest_book['author_name'],
        'Longest Book Pages': longest_book['num_pages']
    }

# create a DataFrame for the years you want to analyze (e.g., from 2010 to 2020)
years = list(range(2010, 2021))
year_summaries = [year_summary(year) for year in years]
year_summary_df = pd.DataFrame(year_summaries)

# show the head and tail of the DataFrame
print("Head of the DataFrame (first ten years):")
print(year_summary_df.head(10))
print("\nTail of the DataFrame (last ten years):")
print(year_summary_df.tail(10))


Head of the DataFrame (first ten years):
   Year  Number of Books Published  Total Pages Written  Most Prolific Month  \
0  2010                        324              87326.0                    1   
1  2011                        249              72994.0                   11   
2  2012                        146              42959.0                    1   
3  2013                         99              27776.0                    4   
4  2014                         88              20235.0                    1   
5  2015                        101              24700.0                    2   
6  2016                         65              18371.0                   12   
7  2017                         39              12816.0                    5   
8  2018                         44              12624.0                    2   
9  2019                         29               6629.0                    5   

                                  Longest Book Title    Longest Book Author  \

In [4]:
#RQ3 GPT Code

def year_summary(year):
    year_books = df_books[df_books['publication_date'].dt.year == year]
    num_books = len(year_books)
    total_pages = year_books['num_pages'].sum()
    month_counts = year_books['publication_date'].dt.month.value_counts()
    most_prolific_month = month_counts.idxmax()
    longest_book = year_books.loc[year_books['num_pages'].idxmax()]
    
    return year, num_books, total_pages, most_prolific_month, longest_book['title'], longest_book['author_name'], longest_book['num_pages']

years = list(range(2010, 2021))
year_summary_data = [year_summary(year) for year in years]
year_summary_df = pd.DataFrame(year_summary_data, columns=['Year', 'Number of Books Published', 'Total Pages Written', 'Most Prolific Month', 'Longest Book Title', 'Longest Book Author', 'Longest Book Pages'])

print("Head of the DataFrame (first ten years):")
print(year_summary_df.head(10))
print("\nTail of the DataFrame (last ten years):")
print(year_summary_df.tail(10))


Head of the DataFrame (first ten years):
   Year  Number of Books Published  Total Pages Written  Most Prolific Month  \
0  2010                        324              87326.0                    1   
1  2011                        249              72994.0                   11   
2  2012                        146              42959.0                    1   
3  2013                         99              27776.0                    4   
4  2014                         88              20235.0                    1   
5  2015                        101              24700.0                    2   
6  2016                         65              18371.0                   12   
7  2017                         39              12816.0                    5   
8  2018                         44              12624.0                    2   
9  2019                         29               6629.0                    5   

                                  Longest Book Title    Longest Book Author  \

In [5]:
#Differences of these two codes.

#The first code uses a separate function year_summary to calculate the year's summary and then iterates through the list of years to collect the data.

#The second code combines the calculation and data collection directly within a list comprehension.

#The first code specifies column names when creating the DataFrame using the columns parameter.

#The second code defines column names directly within the list comprehension, which makes it slightly less readable.

In [6]:
#Something Extra

df_books['publication_date'] = pd.to_datetime(df_books['publication_date'], format='%Y-%m-%d', errors='coerce')

def year_summary(year):
    """
    Calculate book summaries for a specific year.

    Args:
        year (int): The year.

    Returns:
        dict: A dictionary containing the year's summary information.
    """
    # Books published in the specified year
    year_books = df_books[df_books['publication_date'].dt.year == year]

    # Number of books published in that year
    num_books_published = len(year_books)

    # Total pages written in that year
    total_pages_written = year_books['num_pages'].sum()

    # Count the number of books published in each month by grouping books by month
    month_counts = year_books['publication_date'].dt.month.value_counts()

    most_prolific_month = month_counts.idxmax() if not month_counts.empty else None

    # Longest book written in that year
    longest_book = year_books.loc[year_books['num_pages'].idxmax()] if not year_books.empty else None

    return {
        'Year': year,
        'Number of Books Published': num_books_published,
        'Total Pages Written': total_pages_written,
        'Most Prolific Month': most_prolific_month,
        'Longest Book Title': longest_book['title'] if longest_book is not None else None,
        'Longest Book Author': longest_book['author_name'] if longest_book is not None else None,
        'Longest Book Pages': longest_book['num_pages'] if longest_book is not None else None
    }

# Get all unique years from the DataFrame
all_years = df_books['publication_date'].dt.year.unique()

# Calculate year summaries for all years
year_summaries = [year_summary(year) for year in all_years]

# Create a DataFrame containing the summaries
year_summary_df = pd.DataFrame(year_summaries)

# Sort the DataFrame by the 'Year' column in ascending order
year_summary_df = year_summary_df.sort_values(by='Year')

# Select the newest 10 years and the oldest 10 years for display
head_table = year_summary_df.tail(10)
tail_table = year_summary_df.head(10)

# Display the newest 10 years table
print("Newest 10 Years:")
print(head_table)

# Display the oldest 10 years table
print("\nOldest 10 Years:")
print(tail_table)


Newest 10 Years:
       Year  Number of Books Published  Total Pages Written  \
45   2013.0                         99              27776.0   
42   2014.0                         88              20235.0   
51   2015.0                        101              24700.0   
72   2016.0                         65              18371.0   
71   2017.0                         39              12816.0   
81   2018.0                         44              12624.0   
48   2019.0                         29               6629.0   
79   2020.0                         22               7347.0   
104  2021.0                          2                772.0   
0       NaN                          0                  0.0   

     Most Prolific Month                                 Longest Book Title  \
45                   4.0       Chess: 5334 Problems, Combinations and Games   
42                   1.0  Business Cycles: A Theoretical, Historical, an...   
51                   2.0    The History of the Russi